In [1]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
conf_dw = {
      'url':'jdbc:mysql://localhost:3306/etlmysql?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigMysql',
      'password':'bigMysql1234@'   
      }
}

conf_dm = {
      'url':'jdbc:mysql://localhost:3306/etlmysqlDM?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigDM',
      'password':'bigDM1234@'   
      }
}

In [4]:
# table 저장 / 반환 관련한 함수 구성
# config : db 연결 정보
# spark.read.jdbc() : db에서 명시된 테이블을 반환받는 모듈
def find_data(config, table_name) :
    return spark.read.jdbc(url= config['url'], table=table_name, properties=config['props'])

def save_data(config, df, table_name) :
    return df.write.jdbc(url= config['url'], table=table_name, mode='overwrite' , properties=config['props'])

In [11]:
restaurant = find_data(conf_dw, 'RESTAURANT_LIST')

In [12]:
restaurant.columns

['addr',
 'food',
 're_rank1',
 're_rank2',
 're_rank3',
 're_rank4',
 're_rank5',
 're_visitor',
 'store_name',
 'tel',
 'id']

In [13]:
restaurant.count()

2054

In [8]:
# restaurant = restaurant.withColumn('id', monotonically_increasing_id())

In [9]:
# restaurant.show()

+--------------------------------+--------+--------------------+-------------------+------------------------+------------------------+-------------------------+-----------------------------------+------------------------------+--------------+---+
|                            addr|    food|            re_rank1|           re_rank2|                re_rank3|                re_rank4|                 re_rank5|                         re_visitor|                    store_name|           tel| id|
+--------------------------------+--------+--------------------+-------------------+------------------------+------------------------+-------------------------+-----------------------------------+------------------------------+--------------+---+
|서울 강동구 상일로6길 39 지하1층|간장게장| 음식이 맛있어요 404|  매장이 넓어요 170|         양이 많아요 141|단체모임 하기 좋아요 131|      가성비가 좋아요 125|게장이 짜지도않고너무 맛있어요부...|                      강동반상|   02-429-2733|  0|
|  서울 강남구 선릉로131길 17 1층|간장게장| 음식이 맛있어요 230|재료가 신선해요 164|     매장이 청결해요 159|       

In [10]:
# restaurant = restaurant.filter(col("addr").startswith("서울"))
# restaurant.count()

2054

## restaurant - dropduplicates

In [14]:
# food와 store_name 컬럼을 통틀어서 중복된 행 찾기
duplicate_rows = restaurant.groupBy("food", "store_name", 'addr').count().filter(col("count") > 1)

# 결과 출력
duplicate_rows.show(truncate=False)

+----+----------+----+-----+
|food|store_name|addr|count|
+----+----------+----+-----+
+----+----------+----+-----+



In [15]:
# 중복 제거
restaurant_drop_duplicate = restaurant.dropDuplicates(["food", "store_name", 'addr'])
restaurant_drop_duplicate.show(truncate=False)

+----------------------------------------------------------------------+--------+--------------------+-------------------+------------------------+------------------------+-------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------+--------------+----+
|addr                                                                  |food    |re_rank1            |re_rank2

In [16]:
# food와 store_name 컬럼을 통틀어서 중복된 행 찾기
duplicate_rows = restaurant_drop_duplicate.groupBy("food", "store_name", 'addr').count().filter(col("count") > 1)

# 결과 출력
duplicate_rows.show(truncate=False)

+----+----------+----+-----+
|food|store_name|addr|count|
+----+----------+----+-----+
+----+----------+----+-----+



In [17]:
# 조건에 맞는 행 필터링
filtered_rows = restaurant_drop_duplicate.filter((col("food") == "감자전") & (col("store_name") == "김창훈포차"))

# 결과 출력
print("Filtered Rows:")
filtered_rows.show(truncate=False)

Filtered Rows:
+----------------------------------------+------+--------------------+----------------------+------------+-----------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------+----+
|addr                                    |food  |re_rank1            |re_rank2              |re_rank3    |re_rank4         |re_rank5             |re_visitor                                                                                                                                                                        |store_name|tel           |id  |
+----------------------------------------+------+--------------------+----------------------+------------+-----------------+---------------------+-----------------------------------------------------------------------------------------------------------------------------

In [18]:
restaurant_drop_duplicate.count()

2054

## VISIOTR_REVIEW tbl

In [19]:
selected_columns = ['id', 're_visitor']
visitor_review = restaurant.select(*selected_columns)
visitor_review.show()
visitor_review.count()

+----+-----------------------------------+
|  id|                         re_visitor|
+----+-----------------------------------+
|1088|게장이 짜지도않고너무 맛있어요부...|
|1092|  비리지 않고 정갈하게 차려진 게...|
|1105|  간장게장을 좋아하는 우리 딸이 ...|
|1081| 점심시간때 같더니 가격도 할인되...|
|1078| 고미정에 주말 점심시간에 방문해...|
|1098| 정말 오랜만에 먹어본 게장이었어...|
|1110| 홍대간장게장! 간장게장을 처음먹...|
|1107| 작년 여름쯤이었던가.부모님과 경...|
|1101| 삼성동에 유명한 간장게장 집입니...|
|1083|  아빠 칠순이라 저녁식사 했는데 ...|
|1082| 엄마랑 한정식 먹으러 방문했어요...|
|1077|  영등포 최고의 게장집!양념게장 ...|
|1066|   수유 간장게장 맛집이에요!~ 암...|
|1086|  가까운 게장 집을 찾아서 맛있게...|
|1060|  명현만의 간장게장 합정점! 평소...|
|1063| 퇴근후 찾은 서울식물원 저녁메뉴...|
|1062| 방이동 간장게장 최고 맛집이에요...|
|1071|   보는 반찬 아닌 먹는 반찬으로 ...|
|1085| 가성비는 좋아요직원 분은 바빠서...|
|1097|   명불허전 밥도둑 간장게장.. 너...|
+----+-----------------------------------+
only showing top 20 rows



2054

In [20]:
visitor_review = visitor_review.withColumnRenamed("id", "res_id")
visitor_review.show()

+------+-----------------------------------+
|res_id|                         re_visitor|
+------+-----------------------------------+
|  1088|게장이 짜지도않고너무 맛있어요부...|
|  1092|  비리지 않고 정갈하게 차려진 게...|
|  1105|  간장게장을 좋아하는 우리 딸이 ...|
|  1081| 점심시간때 같더니 가격도 할인되...|
|  1078| 고미정에 주말 점심시간에 방문해...|
|  1098| 정말 오랜만에 먹어본 게장이었어...|
|  1110| 홍대간장게장! 간장게장을 처음먹...|
|  1107| 작년 여름쯤이었던가.부모님과 경...|
|  1101| 삼성동에 유명한 간장게장 집입니...|
|  1083|  아빠 칠순이라 저녁식사 했는데 ...|
|  1082| 엄마랑 한정식 먹으러 방문했어요...|
|  1077|  영등포 최고의 게장집!양념게장 ...|
|  1066|   수유 간장게장 맛집이에요!~ 암...|
|  1086|  가까운 게장 집을 찾아서 맛있게...|
|  1060|  명현만의 간장게장 합정점! 평소...|
|  1063| 퇴근후 찾은 서울식물원 저녁메뉴...|
|  1062| 방이동 간장게장 최고 맛집이에요...|
|  1071|   보는 반찬 아닌 먹는 반찬으로 ...|
|  1085| 가성비는 좋아요직원 분은 바빠서...|
|  1097|   명불허전 밥도둑 간장게장.. 너...|
+------+-----------------------------------+
only showing top 20 rows



In [21]:
visitor_review = visitor_review.withColumn('id', monotonically_increasing_id())
visitor_review.show()

+------+-----------------------------------+---+
|res_id|                         re_visitor| id|
+------+-----------------------------------+---+
|  1088|게장이 짜지도않고너무 맛있어요부...|  0|
|  1092|  비리지 않고 정갈하게 차려진 게...|  1|
|  1105|  간장게장을 좋아하는 우리 딸이 ...|  2|
|  1081| 점심시간때 같더니 가격도 할인되...|  3|
|  1078| 고미정에 주말 점심시간에 방문해...|  4|
|  1098| 정말 오랜만에 먹어본 게장이었어...|  5|
|  1110| 홍대간장게장! 간장게장을 처음먹...|  6|
|  1107| 작년 여름쯤이었던가.부모님과 경...|  7|
|  1101| 삼성동에 유명한 간장게장 집입니...|  8|
|  1083|  아빠 칠순이라 저녁식사 했는데 ...|  9|
|  1082| 엄마랑 한정식 먹으러 방문했어요...| 10|
|  1077|  영등포 최고의 게장집!양념게장 ...| 11|
|  1066|   수유 간장게장 맛집이에요!~ 암...| 12|
|  1086|  가까운 게장 집을 찾아서 맛있게...| 13|
|  1060|  명현만의 간장게장 합정점! 평소...| 14|
|  1063| 퇴근후 찾은 서울식물원 저녁메뉴...| 15|
|  1062| 방이동 간장게장 최고 맛집이에요...| 16|
|  1071|   보는 반찬 아닌 먹는 반찬으로 ...| 17|
|  1085| 가성비는 좋아요직원 분은 바빠서...| 18|
|  1097|   명불허전 밥도둑 간장게장.. 너...| 19|
+------+-----------------------------------+---+
only showing top 20 rows



In [22]:
save_data(conf_dm, visitor_review, 'VISITOR_REVIEW2')

## RANK_REVIEW tbl

In [23]:
# 필요한 컬럼 선택하여 새로운 데이터프레임 생성
selected_columns = ['id', 're_rank1', 're_rank2', 're_rank3', 're_rank4', 're_rank5']
rank_review = restaurant.select(*selected_columns)

# 새로운 데이터프레임 출력 또는 다른 작업 수행
rank_review.show()

+----+--------------------+-------------------+------------------------+------------------------+-------------------------+
|  id|            re_rank1|           re_rank2|                re_rank3|                re_rank4|                 re_rank5|
+----+--------------------+-------------------+------------------------+------------------------+-------------------------+
|1088| 음식이 맛있어요 404|  매장이 넓어요 170|         양이 많아요 141|단체모임 하기 좋아요 131|      가성비가 좋아요 125|
|1092| 음식이 맛있어요 230|재료가 신선해요 164|     매장이 청결해요 159|            친절해요 147|  특별한 메뉴가 있어요 80|
|1105| 음식이 맛있어요 117|        친절해요 80|      재료가 신선해요 67| 특별한 메뉴가 있어요 48|       매장이 청결해요 38|
|1081|음식이 맛있어요 1182|       친절해요 697|     재료가 신선해요 672|     매장이 청결해요 601|      가성비가 좋아요 482|
|1078| 음식이 맛있어요 413|       친절해요 157|     재료가 신선해요 155|     가성비가 좋아요 118|  단체모임 하기 좋아요 96|
|1098| 음식이 맛있어요 238|가성비가 좋아요 150|            친절해요 121|     재료가 신선해요 116|          양이 많아요 109|
|1110| 음식이 맛있어요 136| 재료가 신선해요 94|             친절해요 88|          양이 많아요 71|       가

In [24]:
rank_df_1 = rank_review.selectExpr("id", "stack(5, 're_rank1', re_rank1, 're_rank2', re_rank2, 're_rank3', re_rank3, 're_rank4', re_rank4, 're_rank5', re_rank5) as (rank, value)")
rank_df_1.show(truncate=False)

+----+--------+------------------------+
|id  |rank    |value                   |
+----+--------+------------------------+
|1088|re_rank1|음식이 맛있어요 404     |
|1088|re_rank2|매장이 넓어요 170       |
|1088|re_rank3|양이 많아요 141         |
|1088|re_rank4|단체모임 하기 좋아요 131|
|1088|re_rank5|가성비가 좋아요 125     |
|1092|re_rank1|음식이 맛있어요 230     |
|1092|re_rank2|재료가 신선해요 164     |
|1092|re_rank3|매장이 청결해요 159     |
|1092|re_rank4|친절해요 147            |
|1092|re_rank5|특별한 메뉴가 있어요 80 |
|1105|re_rank1|음식이 맛있어요 117     |
|1105|re_rank2|친절해요 80             |
|1105|re_rank3|재료가 신선해요 67      |
|1105|re_rank4|특별한 메뉴가 있어요 48 |
|1105|re_rank5|매장이 청결해요 38      |
|1081|re_rank1|음식이 맛있어요 1182    |
|1081|re_rank2|친절해요 697            |
|1081|re_rank3|재료가 신선해요 672     |
|1081|re_rank4|매장이 청결해요 601     |
|1081|re_rank5|가성비가 좋아요 482     |
+----+--------+------------------------+
only showing top 20 rows



In [25]:
rank_df_1 = rank_df_1.drop("rank")

# 'value' 컬럼을 기반으로 'text'와 'cnt' 컬럼 생성
rank_df_1 = rank_df_1.withColumn("text", regexp_replace(rank_df_1["value"], r'\d+', ''))
rank_df_1 = rank_df_1.withColumn("cnt", regexp_extract(rank_df_1["value"], r'(\d+)', 1).cast("int"))

# 결과 출력 또는 다른 작업 수행
rank_df_1 = rank_df_1.select("id", "text", "cnt")

rank_df_1.show(truncate=False)

+----+---------------------+----+
|id  |text                 |cnt |
+----+---------------------+----+
|1088|음식이 맛있어요      |404 |
|1088|매장이 넓어요        |170 |
|1088|양이 많아요          |141 |
|1088|단체모임 하기 좋아요 |131 |
|1088|가성비가 좋아요      |125 |
|1092|음식이 맛있어요      |230 |
|1092|재료가 신선해요      |164 |
|1092|매장이 청결해요      |159 |
|1092|친절해요             |147 |
|1092|특별한 메뉴가 있어요 |80  |
|1105|음식이 맛있어요      |117 |
|1105|친절해요             |80  |
|1105|재료가 신선해요      |67  |
|1105|특별한 메뉴가 있어요 |48  |
|1105|매장이 청결해요      |38  |
|1081|음식이 맛있어요      |1182|
|1081|친절해요             |697 |
|1081|재료가 신선해요      |672 |
|1081|매장이 청결해요      |601 |
|1081|가성비가 좋아요      |482 |
+----+---------------------+----+
only showing top 20 rows



In [26]:
rank_df_1.count()

10270

## CODEZIP tbl

In [27]:
unique_texts = rank_df_1.select("text").distinct()

# 결과 출력
unique_texts.show(truncate=False)
unique_texts.count()

+---------------------+
|text                 |
+---------------------+
|품질이 좋아요        |
|종류가 다양해요      |
|위생적이에요         |
|기본 안주가 좋아요   |
|건강한 맛이에요      |
|특별한 메뉴가 있어요 |
|단체모임 하기 좋아요 |
|혼술하기 좋아요      |
|화장실이 깨끗해요    |
|포장이 깔끔해요      |
|메뉴 구성이 알차요   |
|매장이 청결해요      |
|비싼 만큼 가치있어요 |
|재료가 신선해요      |
|주차하기 편해요      |
|음악이 좋아요        |
|컨셉이 독특해요      |
|음식이 맛있어요      |
|미등록               |
|음식이 빨리 나와요   |
+---------------------+
only showing top 20 rows



36

In [28]:
# rank_df_1에서 distinct한 text 컬럼 값 추출
distinct_texts = rank_df_1.select("text").distinct()

# monotonically_increasing_id 함수를 사용하여 각 행에 고유한 ID 부여
distinct_texts = distinct_texts.withColumn("code", monotonically_increasing_id())

# distinct_texts를 사용하여 codezip 테이블 생성
codezip = distinct_texts.select("code", "text")

# 결과 출력 또는 다른 작업 수행
codezip.show(truncate=False)
codezip.count()

+----+---------------------+
|code|text                 |
+----+---------------------+
|0   |품질이 좋아요        |
|1   |종류가 다양해요      |
|2   |위생적이에요         |
|3   |기본 안주가 좋아요   |
|4   |건강한 맛이에요      |
|5   |특별한 메뉴가 있어요 |
|6   |단체모임 하기 좋아요 |
|7   |혼술하기 좋아요      |
|8   |화장실이 깨끗해요    |
|9   |포장이 깔끔해요      |
|10  |메뉴 구성이 알차요   |
|11  |매장이 청결해요      |
|12  |비싼 만큼 가치있어요 |
|13  |재료가 신선해요      |
|14  |주차하기 편해요      |
|15  |음악이 좋아요        |
|16  |컨셉이 독특해요      |
|17  |음식이 맛있어요      |
|18  |미등록               |
|19  |음식이 빨리 나와요   |
+----+---------------------+
only showing top 20 rows



36

## RANK_REVIEW tbl 'text' CODEZIP 'code'로 변환

In [29]:
# 'store_name', 'text' 컬럼을 기준으로 조인
rank_review = rank_df_1.join(codezip, ["text"])

# 결과 출력 또는 다른 작업 수행
rank_review.show(truncate=False)
rank_review.count()

+-------------------+----+---+----+
|text               |id  |cnt|code|
+-------------------+----+---+----+
|품질이 좋아요      |2019|12 |0   |
|종류가 다양해요    |2019|5  |1   |
|종류가 다양해요    |2021|71 |1   |
|종류가 다양해요    |2050|30 |1   |
|종류가 다양해요    |2038|12 |1   |
|종류가 다양해요    |2036|193|1   |
|종류가 다양해요    |2045|10 |1   |
|종류가 다양해요    |2043|10 |1   |
|위생적이에요       |2021|62 |2   |
|위생적이에요       |2038|8  |2   |
|위생적이에요       |2048|4  |2   |
|기본 안주가 좋아요 |1266|143|3   |
|기본 안주가 좋아요 |1219|21 |3   |
|기본 안주가 좋아요 |1220|843|3   |
|기본 안주가 좋아요 |1262|97 |3   |
|기본 안주가 좋아요 |1260|297|3   |
|기본 안주가 좋아요 |1254|139|3   |
|기본 안주가 좋아요 |1249|631|3   |
|기본 안주가 좋아요 |1264|28 |3   |
|기본 안주가 좋아요 |1253|174|3   |
+-------------------+----+---+----+
only showing top 20 rows



10270

In [30]:
rank_review = rank_review.drop('text')
rank_review.show()
rank_review.count()

+----+---+----+
|  id|cnt|code|
+----+---+----+
|2019| 12|   0|
|2019|  5|   1|
|2021| 71|   1|
|2050| 30|   1|
|2038| 12|   1|
|2036|193|   1|
|2045| 10|   1|
|2043| 10|   1|
|2021| 62|   2|
|2038|  8|   2|
|2048|  4|   2|
|1266|143|   3|
|1219| 21|   3|
|1220|843|   3|
|1262| 97|   3|
|1260|297|   3|
|1254|139|   3|
|1249|631|   3|
|1264| 28|   3|
|1253|174|   3|
+----+---+----+
only showing top 20 rows



10270

In [31]:
rank_review = rank_review.withColumnRenamed("id", "res_id")
rank_review = rank_review.withColumn('id', monotonically_increasing_id())
rank_review.show()
rank_review.count()

+------+---+----+---+
|res_id|cnt|code| id|
+------+---+----+---+
|  2019| 12|   0|  0|
|  2019|  5|   1|  1|
|  2021| 71|   1|  2|
|  2050| 30|   1|  3|
|  2038| 12|   1|  4|
|  2036|193|   1|  5|
|  2045| 10|   1|  6|
|  2043| 10|   1|  7|
|  2021| 62|   2|  8|
|  2038|  8|   2|  9|
|  2048|  4|   2| 10|
|  1266|143|   3| 11|
|  1219| 21|   3| 12|
|  1220|843|   3| 13|
|  1262| 97|   3| 14|
|  1260|297|   3| 15|
|  1254|139|   3| 16|
|  1249|631|   3| 17|
|  1264| 28|   3| 18|
|  1253|174|   3| 19|
+------+---+----+---+
only showing top 20 rows



10270

In [18]:
save_data(conf_dm, restaurant_drop_duplicate, 'RESTAURANT_INFO')

In [32]:
save_data(conf_dm, codezip, 'CODEZIP2')

In [33]:
save_data(conf_dm, rank_review, 'RANK_REVIEW2')